In [81]:
import pandas as pd
from feature_engine.datetime import DatetimeFeatures
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import os
import pickle
import numpy as np
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [82]:
import pandas as pd
from feature_engine.encoding import MeanEncoder, OneHotEncoder, OrdinalEncoder
from feature_engine.imputation import (
    ArbitraryNumberImputer,
    CategoricalImputer,
    MeanMedianImputer,
)
from feature_engine.selection import DropConstantFeatures
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

In [83]:
CATEGORICAL_COLUMNS = ["holiday_name"]
NUMERICAL_COLUMNS = [
    "holiday",
    "shutdown",
    "mini_shutdown",
    "shops_closed",
    "winter_school_holidays",
    "school_holidays",
    "blackout",
    "mov_change",
    "frankfurt_shutdown",
    # "date_year",
    # "date_month",
    # "date_day_of_month",
    # "date_day_of_week",
    # "date_day_of_year",
    # "date_weekend",
]
DATE_COLUMNS = ["date"]
FEATRURE_COLUMNS = NUMERICAL_COLUMNS + CATEGORICAL_COLUMNS + DATE_COLUMNS
TARGET_COLUMNS = "orders"
TRAIN_LEVEL = ["warehouse"]
MODEL_PATH = "model_registry"

# 1. Preprocessing


In [84]:
# load the data
PATH = "./data/"
df_train = pd.read_csv(f"{PATH}train.csv", parse_dates=["date"])
df_test = pd.read_csv(f"{PATH}test.csv", parse_dates=["date"])
# df_calender = pd.read_csv(f'{PATH}train_calendar.csv')
# df["date"] = pd.to_datetime(df["date"])
columns_to_integer = [
    "holiday",
    "shutdown",
    "mini_shutdown",
    "shops_closed",
    "winter_school_holidays",
    "school_holidays",
    "blackout",
    "mov_change",
    "frankfurt_shutdown",
]
df_train[columns_to_integer] = df_train[columns_to_integer].astype(int)
df_train.drop(columns=["user_activity_1", "user_activity_2"], inplace=True)
columns_to_integer_test = [
    "holiday",
    "shops_closed",
    "winter_school_holidays",
    "school_holidays",
]
df_test[columns_to_integer_test] = df_test[columns_to_integer_test].astype(int)

In [85]:
df_train.warehouse.unique()

array(['Prague_1', 'Brno_1', 'Prague_2', 'Prague_3', 'Munich_1',
       'Frankfurt_1', 'Budapest_1'], dtype=object)

In [86]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7340 entries, 0 to 7339
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   warehouse               7340 non-null   object        
 1   date                    7340 non-null   datetime64[ns]
 2   orders                  7340 non-null   float64       
 3   holiday_name            218 non-null    object        
 4   holiday                 7340 non-null   int64         
 5   shutdown                7340 non-null   int64         
 6   mini_shutdown           7340 non-null   int64         
 7   shops_closed            7340 non-null   int64         
 8   winter_school_holidays  7340 non-null   int64         
 9   school_holidays         7340 non-null   int64         
 10  blackout                7340 non-null   int64         
 11  mov_change              7340 non-null   int64         
 12  frankfurt_shutdown      7340 non-null   int64   

In [87]:
def missing_dates(df, warehouse, binary_columns=None):
    df1 = df.query(f'warehouse == "{warehouse}"')
    df1 = df1.set_index("date").asfreq("D")
    df1["missing"] = df1["warehouse"].isnull().astype(int)
    df1["warehouse"] = df1["warehouse"].fillna(warehouse)
    df1[binary_columns] = df1[binary_columns].fillna(0).astype(int)
    df1["id"] = df1["id"].fillna(df1["warehouse"] + "_" + df1.index.astype(str))
    return df1

In [88]:
# correct train data
df_train_new = pd.DataFrame()
for warehouse in df_train["warehouse"].unique():
    df1 = missing_dates(
        df_train,
        warehouse,
        binary_columns=[
            "holiday",
            "shutdown",
            "mini_shutdown",
            "shops_closed",
            "winter_school_holidays",
            "school_holidays",
            "blackout",
            "mov_change",
            "frankfurt_shutdown",
            "precipitation",
            "snow",
        ],
    )
    df_train_new = pd.concat([df_train_new, df1])


# df_train_new.head()
df_train_new = df_train_new.reset_index()
df_train_new["holiday_name"] = np.where(df_train_new["holiday_name"].isnull(), 'no_holiday', df_train_new["holiday_name"])
# df_train_new = df_train_new.query("missing == 0")

# # df_train_new.query("missing == 1")
# print(df_train_new.shape)
# print(df_train.shape)
# # missing_dates(df_train, "Prague_1")

In [89]:
df_train_new

,date,warehouse,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,id,missing
0,2020-12-05,Prague_1,6895.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-05,0
1,2020-12-06,Prague_1,6584.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-06,0
2,2020-12-07,Prague_1,7030.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-07,0
3,2020-12-08,Prague_1,6550.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-08,0
4,2020-12-09,Prague_1,6910.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-09,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705,2024-03-10,Budapest_1,6733.0,no_holiday,0,0,0,0,0,0,0,0,0,1,0,Budapest_1_2024-03-10,0
7706,2024-03-11,Budapest_1,6492.0,no_holiday,0,0,0,0,0,0,0,0,0,1,0,Budapest_1_2024-03-11,0
7707,2024-03-12,Budapest_1,6661.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Budapest_1_2024-03-12,0
7708,2024-03-13,Budapest_1,6843.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Budapest_1_2024-03-13,0


In [90]:
df_train_new.head()

,date,warehouse,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,id,missing
0,2020-12-05,Prague_1,6895.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-05,0
1,2020-12-06,Prague_1,6584.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-06,0
2,2020-12-07,Prague_1,7030.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-07,0
3,2020-12-08,Prague_1,6550.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-08,0
4,2020-12-09,Prague_1,6910.0,no_holiday,0,0,0,0,0,0,0,0,0,0,0,Prague_1_2020-12-09,0


In [91]:
# correct test data
df_test_new = pd.DataFrame()
for warehouse in df_test["warehouse"].unique():
    df1 = missing_dates(
        df_test,
        warehouse,
        binary_columns=[
            "holiday",
            "shops_closed",
            "winter_school_holidays",
            "school_holidays",
        ],
    )
    df_test_new = pd.concat([df_test_new, df1])

df_test_new = df_test_new.reset_index()

cols_to_be_added = [
    col
    for col in df_train_new.columns
    if col not in df_test_new.columns
    and col not in ["orders", "user_activity_1", "user_activity_2"]
]

# add missing columns to test data
for col in cols_to_be_added:
    df_test_new[col] = 0

df_test_new[TARGET_COLUMNS] = np.nan
df_test_new["holiday_name"] = np.where(df_test_new["holiday_name"].isnull(), 'no_holiday', df_test_new["holiday_name"])


# df_test_new = df_test_new.query("missing == 0")

# # df_test_new.head()
# print(df_test_new.shape)
# print(df_test.shape)

In [92]:
df_train_new["holiday_name"].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
7705    False
7706    False
7707    False
7708    False
7709    False
Name: holiday_name, Length: 7710, dtype: bool

In [93]:
df_test_new.head()

,date,warehouse,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id,missing,shutdown,mini_shutdown,blackout,mov_change,frankfurt_shutdown,precipitation,snow,orders
0,2024-03-16,Prague_1,no_holiday,0,0,0,0,Prague_1_2024-03-16,0,0,0,0,0,0,0,0,NaN
1,2024-03-17,Prague_1,no_holiday,0,0,0,0,Prague_1_2024-03-17,0,0,0,0,0,0,0,0,NaN
2,2024-03-18,Prague_1,no_holiday,0,0,0,0,Prague_1_2024-03-18,0,0,0,0,0,0,0,0,NaN
3,2024-03-19,Prague_1,no_holiday,0,0,0,0,Prague_1_2024-03-19,0,0,0,0,0,0,0,0,NaN
4,2024-03-20,Prague_1,no_holiday,0,0,0,0,Prague_1_2024-03-20,0,0,0,0,0,0,0,0,NaN


# 2. Feature Engineering


In [94]:
transformer_date = DatetimeFeatures(
                variables=DATE_COLUMNS,
                features_to_extract=[
                    "year",
                    "month",
                    "day_of_month",
                    "day_of_week",
                    "day_of_month",
                    "day_of_year",
                    "weekend",
                ],
                drop_original=False,
            )

df_train_new = transformer_date.fit_transform(df_train_new)
df_test_new = transformer_date.transform(df_test_new)

# rename columns and remove date_XXX to XXX
df_train_new.columns = df_train_new.columns.str.replace("date_", "")
df_test_new.columns = df_test_new.columns.str.replace("date_", "")

df_train_new.to_csv(f"{PATH}train_new.csv", index=False)
df_test_new.to_csv(f"{PATH}test_new.csv", index=False)
df_train_new

,date,warehouse,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,...,precipitation,snow,id,missing,year,month,day_of_month,day_of_week,day_of_year,weekend
0,2020-12-05,Prague_1,6895.0,no_holiday,0,0,0,0,0,0,...,0,0,Prague_1_2020-12-05,0,2020,12,5,5,340,1
1,2020-12-06,Prague_1,6584.0,no_holiday,0,0,0,0,0,0,...,0,0,Prague_1_2020-12-06,0,2020,12,6,6,341,1
2,2020-12-07,Prague_1,7030.0,no_holiday,0,0,0,0,0,0,...,0,0,Prague_1_2020-12-07,0,2020,12,7,0,342,0
3,2020-12-08,Prague_1,6550.0,no_holiday,0,0,0,0,0,0,...,0,0,Prague_1_2020-12-08,0,2020,12,8,1,343,0
4,2020-12-09,Prague_1,6910.0,no_holiday,0,0,0,0,0,0,...,0,0,Prague_1_2020-12-09,0,2020,12,9,2,344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705,2024-03-10,Budapest_1,6733.0,no_holiday,0,0,0,0,0,0,...,1,0,Budapest_1_2024-03-10,0,2024,3,10,6,70,1
7706,2024-03-11,Budapest_1,6492.0,no_holiday,0,0,0,0,0,0,...,1,0,Budapest_1_2024-03-11,0,2024,3,11,0,71,0
7707,2024-03-12,Budapest_1,6661.0,no_holiday,0,0,0,0,0,0,...,0,0,Budapest_1_2024-03-12,0,2024,3,12,1,72,0
7708,2024-03-13,Budapest_1,6843.0,no_holiday,0,0,0,0,0,0,...,0,0,Budapest_1_2024-03-13,0,2024,3,13,2,73,0


In [ ]:
df_train_new = df_train_new.query("missing == 0")

# df_train_new.query("missing == 1")
print(df_train_new.shape)
print(df_train.shape)
# missing_dates(df_train, "Prague_1")

In [ ]:
df_test_new = df_test_new.query("missing == 0")

# df_test_new.head()
print(df_test_new.shape)
print(df_test.shape)

# 3. Model Training


In [52]:
pipeline = Pipeline(
    [
        # Impute missing categorical except mean-encoded ones, normally happens in the test data
        (
            "missing_categoricals",
            CategoricalImputer(
                imputation_method="missing", variables=CATEGORICAL_COLUMNS
            ),
        ),
        # Impute 0 for missing numericals
        (
            "missing_numerical",
            ArbitraryNumberImputer(variables=NUMERICAL_COLUMNS, arbitrary_number=0),
        ),
        (
            "date_features",
            DatetimeFeatures(
                variables=DATE_COLUMNS,
                features_to_extract=[
                    "year",
                    "month",
                    "day_of_month",
                    "day_of_week",
                    "day_of_month",
                    "day_of_year",
                    "weekend",
                ],
                drop_original=True,
            ),
        ),
        # OneHotEncode the rest of categorical
        (
            "onehot_encoding",
            OneHotEncoder(
                top_categories=6,
                variables=CATEGORICAL_COLUMNS,
                ignore_format=True,
            ),
        ),
        # Drop Constant features
        ("drop_constant", DropConstantFeatures(tol=1)),
        ("xgb", XGBRegressor(n_estimators=100, random_state=42)),
    ]
)

In [53]:
def train_model_core(
    df: pd.DataFrame,
    train_level: list,
    feature_columns: list,
    target_column: str,
    pipeline: Pipeline,
    model_dir: str = "model_registry",
) -> pd.DataFrame:
    """This function run the train for a single level.

    Args:
        df (pd.DataFrame): A dataframe including level columns, features, and target columns
        train_level (list): training level i.e. `['warehouse', 'item_class']`
        feature_columns (list): Feature columns.
        target_column (str): traget columns should be a single string.
        model_dir (str, optional): Name of the directory for saving the model pickel files. Defaults to "model_registry".

    Returns:
        pd.DataFrame: A single row dataframe including the train level, model registry path, and error if we have any.
    """

    e = ""
    train_level_value = df[train_level].iloc[0, :].to_list()

    try:
        X = df[feature_columns]
        y = df[target_column]

        model = pipeline.fit(X, y)

        # saving the model to model registry
        os.makedirs(model_dir, exist_ok=True)
        model_string = "_".join(
            [f"{a}_{b}" for a, b in zip(train_level, train_level_value)]
        )
        model_path = os.path.join(model_dir, f"model_{model_string}.pkl")
        with open(model_path, "wb") as f:
            pickle.dump(model, f)

    except Exception as e:
        print("e")

    train_df = pd.DataFrame(
        [train_level_value + ["_".join(train_level)] + [model_path] + [e]],
        columns=[*train_level, "train_level", "model_path", "error"],
    )

    return train_df

In [54]:
def score_model_core(
    df: pd.DataFrame,
    train_level: list,
    feature_columns: list,
    target_column: str,
    model_dir: str = "model_registry",
) -> pd.DataFrame:
    """Loading a model from model registry and inference.

    Args:
        df (pd.DataFrame): A dataframe including level columns, features, and target columns
        train_level (list): training level i.e. `['warehouse', 'item_class']`
        feature_columns (list): Feature columns.
        target_column (str): traget columns should be a single string.
        model_dir (str, optional): Name of the directory for saving the model pickel files. Defaults to "model_registry".

    Returns:
        pd.DataFrame: A dataframe including all input dataframe with forecast and error column added at the end.
    """
    e = ""
    df = df.copy()
    train_level_value = df[train_level].iloc[0, :].to_list()

    try:
        X = df[feature_columns]
        y = df[target_column]

        model_string = "_".join(
            [f"{a}_{b}" for a, b in zip(train_level, train_level_value)]
        )
        model_path = os.path.join(model_dir, f"model_{model_string}.pkl")
        with open(model_path, "rb") as f:
            model = pickle.load(f)

        df.loc[:, "forecast"] = model.predict(X)
        df.loc[:, "model_path"] = model_path

    except Exception as e:
        df.loc[:, "forecast"] = np.null
        df.loc[:, "model_path"] = np.null

    df.loc[:, "error"] = e
    return df

## 3.1. Single Model Testing


In [55]:
warehouse = "Prague_1"
_df = df_train_new.query(f"warehouse== '{warehouse}'")
_df2 = df_test_new.query(f"warehouse== '{warehouse}'")

In [56]:
train_model_core(
    _df, TRAIN_LEVEL, FEATRURE_COLUMNS, TARGET_COLUMNS, pipeline, model_dir=MODEL_PATH
)

,warehouse,train_level,model_path,error
0,Prague_1,warehouse,model_registry\model_warehouse_Prague_1.pkl,


In [57]:
_df2.isnull().sum()

date                       0
warehouse                  0
holiday_name              57
holiday                    0
shops_closed               0
winter_school_holidays     0
school_holidays            0
id                         0
missing                    0
shutdown                   0
mini_shutdown              0
blackout                   0
mov_change                 0
frankfurt_shutdown         0
precipitation              0
snow                       0
orders                    61
dtype: int64

In [58]:
score_model_core(
    _df2, TRAIN_LEVEL, FEATRURE_COLUMNS, TARGET_COLUMNS, model_dir=MODEL_PATH
)

,date,warehouse,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id,missing,shutdown,mini_shutdown,blackout,mov_change,frankfurt_shutdown,precipitation,snow,orders,forecast,model_path,error
0,2024-03-16,Prague_1,NaN,0,0,0,0,Prague_1_2024-03-16,0,0,0,0,0,0,0,0,NaN,10271.309570,model_registry\model_warehouse_Prague_1.pkl,
1,2024-03-17,Prague_1,NaN,0,0,0,0,Prague_1_2024-03-17,0,0,0,0,0,0,0,0,NaN,10328.383789,model_registry\model_warehouse_Prague_1.pkl,
2,2024-03-18,Prague_1,NaN,0,0,0,0,Prague_1_2024-03-18,0,0,0,0,0,0,0,0,NaN,9720.911133,model_registry\model_warehouse_Prague_1.pkl,
3,2024-03-19,Prague_1,NaN,0,0,0,0,Prague_1_2024-03-19,0,0,0,0,0,0,0,0,NaN,9441.130859,model_registry\model_warehouse_Prague_1.pkl,
4,2024-03-20,Prague_1,NaN,0,0,0,0,Prague_1_2024-03-20,0,0,0,0,0,0,0,0,NaN,9291.542969,model_registry\model_warehouse_Prague_1.pkl,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2024-05-11,Prague_1,NaN,0,0,0,0,Prague_1_2024-05-11,0,0,0,0,0,0,0,0,NaN,10292.479492,model_registry\model_warehouse_Prague_1.pkl,
57,2024-05-12,Prague_1,NaN,0,0,0,0,Prague_1_2024-05-12,0,0,0,0,0,0,0,0,NaN,10035.849609,model_registry\model_warehouse_Prague_1.pkl,
58,2024-05-13,Prague_1,NaN,0,0,0,0,Prague_1_2024-05-13,0,0,0,0,0,0,0,0,NaN,9903.984375,model_registry\model_warehouse_Prague_1.pkl,
59,2024-05-14,Prague_1,NaN,0,0,0,0,Prague_1_2024-05-14,0,0,0,0,0,0,0,0,NaN,9321.391602,model_registry\model_warehouse_Prague_1.pkl,


## 3.2. Batch Model Training


In [59]:
df_train_new.groupby(TRAIN_LEVEL).apply(
    train_model_core,
    TRAIN_LEVEL,
    FEATRURE_COLUMNS,
    TARGET_COLUMNS,
    pipeline,
    model_dir=MODEL_PATH,
    include_groups=True,
).reset_index(drop=True)

,warehouse,train_level,model_path,error
0,Brno_1,warehouse,model_registry\model_warehouse_Brno_1.pkl,
1,Budapest_1,warehouse,model_registry\model_warehouse_Budapest_1.pkl,
2,Frankfurt_1,warehouse,model_registry\model_warehouse_Frankfurt_1.pkl,
3,Munich_1,warehouse,model_registry\model_warehouse_Munich_1.pkl,
4,Prague_1,warehouse,model_registry\model_warehouse_Prague_1.pkl,
5,Prague_2,warehouse,model_registry\model_warehouse_Prague_2.pkl,
6,Prague_3,warehouse,model_registry\model_warehouse_Prague_3.pkl,


In [60]:
df_test_new.groupby(TRAIN_LEVEL).apply(
    score_model_core,
    TRAIN_LEVEL,
    FEATRURE_COLUMNS,
    TARGET_COLUMNS,
    model_dir=MODEL_PATH,
    include_groups=True,
).reset_index(drop=True)

,date,warehouse,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id,missing,shutdown,mini_shutdown,blackout,mov_change,frankfurt_shutdown,precipitation,snow,orders,forecast,model_path,error
0,2024-03-16,Brno_1,NaN,0,0,0,0,Brno_1_2024-03-16,0,0,0,0,0,0,0,0,NaN,9377.536133,model_registry\model_warehouse_Brno_1.pkl,
1,2024-03-17,Brno_1,NaN,0,0,0,0,Brno_1_2024-03-17,0,0,0,0,0,0,0,0,NaN,8303.705078,model_registry\model_warehouse_Brno_1.pkl,
2,2024-03-18,Brno_1,NaN,0,0,0,0,Brno_1_2024-03-18,0,0,0,0,0,0,0,0,NaN,8439.435547,model_registry\model_warehouse_Brno_1.pkl,
3,2024-03-19,Brno_1,NaN,0,0,0,0,Brno_1_2024-03-19,0,0,0,0,0,0,0,0,NaN,8443.680664,model_registry\model_warehouse_Brno_1.pkl,
4,2024-03-20,Brno_1,NaN,0,0,0,0,Brno_1_2024-03-20,0,0,0,0,0,0,0,0,NaN,8422.693359,model_registry\model_warehouse_Brno_1.pkl,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,2024-05-11,Prague_3,NaN,0,0,0,0,Prague_3_2024-05-11,0,0,0,0,0,0,0,0,NaN,5515.541016,model_registry\model_warehouse_Prague_3.pkl,
393,2024-05-12,Prague_3,NaN,0,0,0,0,Prague_3_2024-05-12,0,0,0,0,0,0,0,0,NaN,5364.634766,model_registry\model_warehouse_Prague_3.pkl,
394,2024-05-13,Prague_3,NaN,0,0,0,0,Prague_3_2024-05-13,0,0,0,0,0,0,0,0,NaN,5219.994141,model_registry\model_warehouse_Prague_3.pkl,
395,2024-05-14,Prague_3,NaN,0,0,0,0,Prague_3_2024-05-14,0,0,0,0,0,0,0,0,NaN,5173.205566,model_registry\model_warehouse_Prague_3.pkl,
